In [1]:
import numpy as np

def TRADING_RULE(P_hat, P, options):
    if options == 'BUY HOLD':
        return TRADING_RULE_1(P)
    elif options == 'LONG SHORT':
        return TRADING_RULE_2(P_hat, P)
    elif options == 'DAY TRADE':
        return TRADING_RULE_3(P_hat, P)

def TRADING_RULE_1(P):
    T = P.shape[0]
    signal = np.zeros(T)
    signal[0] = 1
    signal[-1] = -1
    return signal

def TRADING_RULE_2(P_hat, P):
    T = P.shape[0]
    signal = np.zeros(T)
    for t in range(1, T):
        if (P_hat[t-1] > P[t-1]) + (P_hat[t] < P[t]) == 2:  # FORCAST > OPEN: LONG
            signal[t] = 1
        elif (P_hat[t-1] < P[t-1]) + (P_hat[t] > P[t]) == 2:  # FORECAST < OPEN: SHORT
            signal[t] = -1
    return signal

def TRADING_RULE_3(P_hat, P):
    T = P.shape[0]
    signal = np.zeros(T)
    for t in range(1, T):
        if P_hat[t] > P[t]:  # FORCAST > OPEN: LONG
            signal[t] = 1
        elif P_hat[t] < P[t]:  # FORECAST < OPEN: SHORT
            signal[t] = -1
    return signal

In [2]:
import numpy as np

def ACCOUNT(signal, price, balance0, options):
    if options == 'BUY HOLD':
        position, value = BUY_HOLD(signal, price, balance0)
    elif options == 'LONG ONLY':
        position, value = LongOnly(signal, price, balance0)
    elif options == 'LONG SHORT':
        position, value = LongShort(signal, price, balance0)
    elif options == 'DAY TRADE':
        position, value = DayTrade(signal, price, balance0)
    return position, value

def BUY_HOLD(signal, price, balance0):
    position = signal
    value = balance0 - 100 * price['Open'][0] + 100 * price['Close'][-1]
    return position, value

def OrderStatus(position, price, balance0):
    T = len(price)
    N = 100
    OPEN = price['Open'].to_numpy()
    CLOSE = price['Close'].to_numpy()
    value = np.zeros(T)
    t0 = np.where(position != 0)[0][0]  # THE FIRST TRADE
    value[:t0] = balance0
    for t in range(t0, T):
        t2trade, pos_last = np.where(position[:t + 1] != 0)[0][-1], position[np.where(position[:t + 1] != 0)[0][-1]]
        if pos_last == 1:
            value[t] = value[t - 1] - N * OPEN[t2trade] + N * CLOSE[t]
        elif pos_last == -1:
            value[t] = value[t - 1] + N * OPEN[t2trade] - N * CLOSE[t]
    return value

def LongOnly(signal, price, balance0):
    T = len(signal)
    position = np.zeros(T)
    t = 0
    while t < T:
        time2buy = np.where(signal[t:] == 1)[0]  # the first buy
        if len(time2buy) == 0:
            t = T
        else:
            t += time2buy[0]
            position[t] = 1  # create a buy position
            t += 1
            time2sell = np.where(signal[t:] == -1)[0]  # the first sell
            if len(time2sell) == 0:
                t = T
            else:
                t += time2sell[0]
                position[t] = -1  # create a sell position
                t += 1
    value = OrderStatus(position, price, balance0)
    return position, value

def LongShort(signal, price, balance0):
    T = len(signal)
    position = np.zeros(T)
    t = 0
    while t < T:
        time2trade = np.where(signal[t:] != 0)[0]  # the first position
        if len(time2trade) == 0:
            t = T
        else:
            t += time2trade[0]
            if signal[t] == 1:  # buy
                position[t] = 1
                t1 = np.where(signal[t:] == -1)[0]
                t2 = np.where(price['Open'][t:] <= (1 - 0.20) * price['Close'][t - time2trade[0]])[0]
                time2sell = min([t1[0], t2[0]])
                position[t + time2sell] = -1  # create a sell position
                t += time2sell + 1
            elif signal[t] == -1:  # sell
                position[t] = -1
                t1 = np.where(signal[t:] == 1)[0]
                t2 = np.where(price['Open'][t:] >= (1 + 0.05) * price['Close'][t - time2trade[0]])[0]
                time2buy = min([t1[0], t2[0]])
                position[t + time2buy] = 1  # create a buy position
                t += time2buy + 1
    value = OrderStatus(position, price, balance0)
    return position, value

def DayTrade(signal, price, balance0):
    # signal: NO POSITION OVER NIGHT
    # N: NUMBER OF SHARES BOUGHT OR SOLD
    T = len(signal)
    OPEN = price['Open'].to_numpy()
    CLOSE = price['Close'].to_numpy()
    position = signal
    value = np.zeros(T)
    time2trade = np.where(signal != 0)[0][0]  # 1st TRADE: BUY or SELL
    value[:time2trade] = balance0
    for t in range(time2trade, T):
        N = int(value[t - 1] / OPEN[t])
        if signal[t] == 1:  # BUY AT OPEN, SELL AT CLOSE
            value[t] = value[t - 1] - N * OPEN[t] + N * CLOSE[t]
        elif signal[t] == -1:  # SELL AT OPEN, BUY AT CLOSE
            value[t] = value[t - 1] + N * OPEN[t] - N * CLOSE[t]
    return position, value
